# 4. Modelowanie danych (Modeling)

### Importowanie niezbędnych bibliotek

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

from catboost import CatBoostRegressor

import time
from sklearn.model_selection import GridSearchCV


### Przygotowanie danych

In [2]:
df = pd.read_csv('dane/2023/curr_lct_dl.csv',
                   sep=',',
                   header=0
                  )

# Wybór parametru docelowego (bytes_sec)
target = 'bytes_sec'

# Ekstrakcja cech czasowych
df['dtime'] = pd.to_datetime(df['dtime'])
df['hour'] = df['dtime'].dt.hour
df['day_of_week'] = df['dtime'].dt.dayofweek
df['day_of_month'] = df['dtime'].dt.day


# Usuwanie zbędnych kolumn
df = df.drop(['ddate', 'dtime', 'unit_id'], axis=1)

# Podział na cechy i zmienną docelową
X = df.drop([target], axis=1)
y = df[target]

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transformacje danych
numeric_features = ['packet_size', 
                   'bytes_total', 'duration', 
                   'hour', 'day_of_week', 'day_of_month']

categorical_features = ['target', 'address', 'error_code']

# Definicja preprocessora - przetwarza różne typy cech osobno i łączy wyniki
preprocessor = ColumnTransformer(
    transformers=[
        # Dla cech numerycznych: skalowanie (StandardScaler - średnia=0, odchylenie=1)
        ('num', StandardScaler(), numeric_features),

        # Dla cech kategorycznych: one-hot encoding (tworzy dummy variables)
        # handle_unknown='ignore' - pomija nieznane kategorie (ważne dla danych testowych)
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

Wybranym parametrem  jest *bytes_sec*, który reprezentuje liczbę przesyłanych bajtów na sekundę – miarę przepustowości i wydajności połączenia internetowego. Dane zostały podzielone na dane treningowe (80%) i dane testowe (20%). 

## 1. Random Forest Regressor

Charakterystyka:
* Random Forest to model zespołowy (ensemble), oparty na wielu niezależnych drzewach decyzyjnych.
* Każde drzewo uczy się na losowo wybranym podzbiorze danych oraz cech.
* Końcowa predykcja jest średnią z predykcji wszystkich drzew (regresja).

Zastosowanie:
- Dobrze radzi sobie z danymi liczbowymi oraz (po odpowiednim zakodowaniu) z danymi kategorycznymi.
- Jest odporny na nadmierne dopasowanie (overfitting), szczególnie przy dużej liczbie drzew.
- Umożliwia ocenę ważności cech (feature importance).


Ograniczenia:
- Wymaga jawnego preprocesingu danych kategorycznych (np. OneHotEncoder).
- Może być mniej dokładny niż boosting przy bardzo złożonych zależnościach między cechami.

In [5]:
model_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['packet_size', 'bytes_total',
                                                   'duration', 'hour',
                                                   'day_of_week',
                                                   'day_of_month']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['target', 'address',
                                                   'error_code'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=4, n_estimators=3))])

Dla modelu wyszukiwane są najlepsze parametry z podanych.

W modelu Random Forest Regressor najistotniejsze parametry to:
* Liczba drzew (n_estimators) - Określa, ile drzew decyzyjnych będzie zbudowanych Więcej drzew - większa stabilność i dokładność, ale wolniejsze obliczenia. Zbyt mało drzew - model może być niedostatecznie dokładny. Typowe wartości: 50–500 (zależnie od rozmiaru danych).
* Głębokość drzew (max_depth) - Kontroluje maksymalną głębokość każdego drzewa. Zbyt duża wartość - ryzyko overfittingu (drzewa zapamiętują szum w danych). Zbyt mała - underfitting (model zbyt uproszczony). Typowe wartości: 3–20 (lub None dla pełnej głębokości, ale ryzyko overfitu).

In [4]:
param_grid = {
    'regressor__n_estimators': [10, 20, 50],
    'regressor__max_depth': [3, 5, 10]
}

grid = GridSearchCV(model_rf, param_grid, cv=3, scoring='r2')
grid.fit(X_train, y_train)

model_rf = grid.best_estimator_
print(f"Najlepsze wyniki są dla parametrów: {grid.best_params_}")

KeyboardInterrupt: 

## 2. Model XGBoost

Charakterystyka:
* XGBoost (Extreme Gradient Boosting) to wysoce zoptymalizowany model boostingowy.
* Każde kolejne drzewo uczy się błędów popełnionych przez poprzednie.
* Oferuje wsparcie dla regularyzacji i szerokie możliwości dostrajania parametrów.

Zastosowanie:
- Świetnie sprawdza się w wykrywaniu nieliniowych i skomplikowanych relacji między zmiennymi.
- Często stosowany w zawodach typu Kaggle ze względu na wysoką dokładność.
- Obsługuje dużą liczbę danych i cech dzięki optymalizacjom pamięci i CPU.


Ograniczenia:
- Wymaga preprocesingu danych kategorycznych (np. kodowanie One-Hot).
- Może być podatny na przeuczenie przy zbyt głębokich drzewach i braku regularyzacji.

In [6]:
model_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['packet_size', 'bytes_total',
                                                   'duration', 'hour',
                                                   'day_of_week',
                                                   'day_of_month']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['target', 'address',
                                                   'error_code'])])),
                ('regressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsamp...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=3, n_jobs=None,
                              num_parallel_tree=None, ...))])

W modelu XGBoost najistotniejsze parametry to:
* Liczba drzew (n_estimators)
* Głębokość drzew (max_depth)
* Szybkość uczenia (learning_rate) - jak mocno każde nowe drzewo koryguje błędy poprzednich. Niskie wartości (np. 0.01–0.1) - wolniejsze, ale dokładniejsze uczenie. Wysokie wartości (np. >0.3) - szybsze, ale mniej precyzyjne. Typowe wartości: 0.01–0.3. 

In [38]:
param_grid = {
    'regressor__n_estimators': [10, 20, 50],
    'regressor__max_depth': [3, 5, 10],
    'regressor__learning_rate': [0.1, 0.15, 0.3]
}

grid = GridSearchCV(model_xgb, param_grid, cv=3, scoring='r2')
grid.fit(X_train, y_train)

model_xgb = grid.best_estimator_
print(f"Najlepsze wyniki są dla parametrów: {grid.best_params_}")

Najlepsze wyniki są dla parametrów: {'regressor__learning_rate': 0.03, 'regressor__max_depth': 3, 'regressor__n_estimators': 5}


## 3. Model CatBoost Regressor

Charakterystyka:
* CatBoost to model boostingowy stworzony z myślą o danych tablicowych z dużą liczbą cech kategorycznych.
* Automatycznie rozpoznaje i przetwarza cechy kategoryczne, bez potrzeby ich kodowania.
* Oparty na zaawansowanych algorytmach porządkowania danych (ordered boosting).

Zastosowanie:
- Idealny do danych z kolumnami typu „adres”, „kategoria”, „typ błędu” itp.
- Oszczędza czas, ponieważ nie wymaga ręcznego przygotowania danych kategorycznych.
- Umożliwia szybkie testowanie i iterację.


Ograniczenia:
- Może być mniej elastyczny przy integracji z niestandardowymi pipeline’ami scikit-learn.
- Trening może być wolniejszy przy dużych zbiorach danych (w porównaniu do XGBoost).

In [7]:
model_cbr = CatBoostRegressor(
    cat_features=['target', 'address', 'error_code'],
    verbose=0  # Wyłączenie logów
)

W modelu CatBoost Regressor najistotniejsze parametry to:
* Liczba iteracji boostingowych(iterations) - podobny wpływ co liczba drzew w poprzednich przykładach
* Szybkość uczenia (learning_rate)
* Głębokość drzew (max_depth)

In [41]:
param_grid = {
    'iterations': [10, 20, 50],
    'learning_rate': [0.1, 0.15, 0.3],
    'depth': [3, 5, 10]
}

grid = GridSearchCV(model_cbr, param_grid, cv=3, scoring='r2')
grid.fit(X_train, y_train)

model_cbr = grid.best_estimator_
print(f"Najlepsze wyniki są dla parametrów: {grid.best_params_}")

Najlepsze wyniki są dla parametrów: {'depth': 3, 'iterations': 2, 'learning_rate': 0.03}


# Podsumowanie porównawcze

Model | Wymaga preprocesingu | Obsługa danych kategorycznych  | Zalety | Ograniczenia
| --- | --- | --- | --- | --- |
Random Forest | Tak | Nie (po zakodowaniu) | Stabilność, prostota, interpretowalność | Mniej skuteczny w złożonych zależnościach
XGBoost | Tak | Nie (po zakodowaniu) | Wysoka skuteczność, regularyzacja, szybkość działania | Złożony tuning, brak wsparcia dla raw cat
CatBoost | Nie | Tak (natywnie) | Automatyczna obsługa kategorii, wysoka dokładność  | Czas trenowania, mniejsza elastyczność


# 5. Ocena modelu (Evaluation)

Ocena modeli odbywa się przez testowanie ich na przygotowanych w poprzednim punkcie danych *X_test* oraz *y_test*.

In [16]:
models = {
    'Random Forest Regressor': model_rf, 
    'XGBoost': model_xgb,     
    'CatBoost Regressor' : model_cbr,       
}

In [19]:
results = []
for name, model in models.items():
        y_pred = model.predict(X_test)
        
        results.append({
            'Model': name,
            'Mean': y_test.mean(),
            'R2': r2_score(y_test, y_pred)
        })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by=['Model'])
print(results_df.to_string(index=False))

                  Model         Mean       R2
     CatBoost Regressor 3.040139e+07 0.690780
Random Forest Regressor 3.040139e+07 0.723426
                XGBoost 3.040139e+07 0.650447
